In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Imports

In [27]:
import pandas as pd

from get_data import get_data

# Load data

In [18]:
greenfield_df = get_data("q7d6-ambg", params = {"neighbourhood":"GREENFIELD"})

In [19]:
greenfield_df.head()

,account_number,house_number,street_name,garage,neighbourhood_id,neighbourhood,ward,assessed_value,latitude,longitude,point_location,tax_class_pct_1,mill_class_1
0,1089457,11735,40 AVENUE NW,N,5220,GREENFIELD,papastew Ward,295500,53.476048825243744,-113.53718479265814,"{'type': 'Point', 'coordinates': [-113.5371847...",100,RESIDENTIAL
1,1089531,11515,40 AVENUE NW,N,5220,GREENFIELD,papastew Ward,312500,53.476044723369654,-113.5314582622399,"{'type': 'Point', 'coordinates': [-113.5314582...",100,RESIDENTIAL
2,1089572,11511,40 AVENUE NW,N,5220,GREENFIELD,papastew Ward,293500,53.47603283160185,-113.53116714475051,"{'type': 'Point', 'coordinates': [-113.5311671...",100,RESIDENTIAL
3,1089515,11517,40 AVENUE NW,N,5220,GREENFIELD,papastew Ward,285500,53.47604638730065,-113.53160341066878,"{'type': 'Point', 'coordinates': [-113.5316034...",100,RESIDENTIAL
4,1089473,11521,40 AVENUE NW,Y,5220,GREENFIELD,papastew Ward,313000,53.47604889188216,-113.53188256193663,"{'type': 'Point', 'coordinates': [-113.5318825...",100,RESIDENTIAL


In [21]:
greenfield_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   account_number    1000 non-null   object
 1   house_number      991 non-null    object
 2   street_name       991 non-null    object
 3   garage            1000 non-null   object
 4   neighbourhood_id  1000 non-null   object
 5   neighbourhood     1000 non-null   object
 6   ward              1000 non-null   object
 7   assessed_value    1000 non-null   object
 8   latitude          1000 non-null   object
 9   longitude         1000 non-null   object
 10  point_location    1000 non-null   object
 11  tax_class_pct_1   1000 non-null   object
 12  mill_class_1      1000 non-null   object
dtypes: object(13)
memory usage: 101.7+ KB


# Data Cleaning

In [28]:
# convert numeric columns to numbers
numeric_cols = ["account_number",
               "house_number", 
               "neighbourhood_id",
               "assessed_value",
               "latitude",
               "longitude",
               "tax_class_pct_1"]

In [35]:
greenfield_df[numeric_cols] = greenfield_df[numeric_cols].apply(pd.to_numeric)

In [ ]:
greenfield_df.